In [1]:
import numpy as np
import math
import xgboost as xgb
import pandas as pd
import random as rd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from xgboost import plot_importance
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler,StandardScaler,RobustScaler

from sklearn.model_selection import KFold,TimeSeriesSplit
from xgboost import plot_importance
from sklearn.metrics import make_scorer
import time

%matplotlib inline

In [2]:
# load data
train = pd.read_csv("./train.csv", index_col=0)
test = pd.read_csv("./test.csv", index_col=0).reset_index(drop=True)
submission = pd.read_csv("./sample_submission.csv")

In [3]:
train['u-g'] = train['u'] - train['g']
train['g-r'] = train['g'] - train['r']
train['r-i'] = train['r'] - train['i']
train['i-z'] = train['i'] - train['z']
train['dered_u-g'] = train['dered_u'] - train['dered_g']
train['dered_g-r'] = train['dered_g'] - train['dered_r']
train['dered_r-i'] = train['dered_r'] - train['dered_i']
train['dered_i-z'] = train['dered_i'] - train['dered_z']

test['u-g'] = test['u'] - test['g']
test['g-r'] = test['g'] - test['r']
test['r-i'] = test['r'] - test['i']
test['i-z'] = test['i'] - test['z']
test['dered_u-g'] = test['dered_u'] - test['dered_g']
test['dered_g-r'] = test['dered_g'] - test['dered_r']
test['dered_r-i'] = test['dered_r'] - test['dered_i']
test['dered_i-z'] = test['dered_i'] - test['dered_z']

In [4]:
train['r_u-g'] = train['redshift'] * train['u-g']
train['r_g-r'] = train['redshift'] * train['g-r']
train['r_r-i'] = train['redshift'] * train['r-i']
train['r_i-z'] = train['redshift'] * train['i-z']
train['r_dered_u-g'] = train['redshift'] * train['dered_u-g']
train['r_dered_g-r'] = train['redshift'] * train['dered_g-r']
train['r_dered_r-i'] = train['redshift'] * train['dered_r-i']
train['r_dered_i-z'] = train['redshift'] * train['dered_i-z']

test['r_u-g'] = test['redshift'] * test['u-g']
test['r_g-r'] = test['redshift'] * test['g-r']
test['r_r-i'] = test['redshift'] * test['r-i']
test['r_i-z'] = test['redshift'] * test['i-z']
test['r_dered_u-g'] = test['redshift'] * test['dered_u-g']
test['r_dered_g-r'] = test['redshift'] * test['dered_g-r']
test['r_dered_r-i'] = test['redshift'] * test['dered_r-i']
test['r_dered_i-z'] = test['redshift'] * test['dered_i-z']

In [5]:
rare_train = train[["u", "g", "r", "i", "z"]]
dered_train = train[["dered_u", "dered_g", "dered_r", "dered_i", "dered_z"]]

rare_test = test[["u", "g", "r", "i", "z"]]
dered_test = test[["dered_u", "dered_g", "dered_r", "dered_i", "dered_z"]]

In [6]:
train["rare_mean"] = rare_train.mean(axis = 1)
train["rare_var"] = rare_train.var(axis = 1)
train["rare_std"] = rare_train.std(axis = 1)
train["rare_sum"] = rare_train.sum(axis = 1)
train["rare_median"] = rare_train.median(axis = 1)
train["rare_max"] = rare_train.max(axis = 1)
train['dered_mean'] = dered_train.mean(axis=1)
train['dered_var'] = dered_train.var(axis=1)
train['dered_std'] = dered_train.std(axis=1)
train['dered_sum'] = dered_train.sum(axis=1)
train['dered_median'] = dered_train.median(axis=1)
train["dered_max"] = dered_train.max(axis = 1)

test["rare_mean"] = rare_test.mean(axis = 1)
test["rare_var"] = rare_test.var(axis = 1)
test["rare_std"] = rare_test.std(axis = 1)
test["rare_sum"] = rare_test.sum(axis = 1)
test["rare_median"] = rare_test.median(axis = 1)
test["rare_max"] = rare_test.max(axis = 1)
test['dered_mean'] = dered_test.mean(axis=1)
test['dered_var'] = dered_test.var(axis=1)
test['dered_std'] = dered_test.std(axis=1)
test['dered_sum'] = dered_test.sum(axis=1)
test['dered_median'] = dered_test.median(axis=1)
test["dered_max"] = dered_test.max(axis = 1)

In [7]:
# https://classic.sdss.org/education/kron_ARCS.pdf
distance_train = []
for rs in train['redshift']:
    if rs > 0:
        distance_train.append((rs / (1+rs)) * 13.5 * 10**9)
    else:
        distance_train.append((abs(rs-1) / abs(rs)) * 13.5 * 10**9)

train['distance'] = distance_train

In [8]:
# https://classic.sdss.org/education/kron_ARCS.pdf
distance_test = []
for rs in test['redshift']:
    if rs > 0:
        distance_test.append((rs / (1+rs)) * 13.5 * 10**9)
    else:
        distance_test.append((abs(rs-1) / abs(rs)) * 13.5 * 10**9)

test['distance'] = distance_test

In [9]:
# n값 처리
train['nO-nD'] = train['nObserve'] - train['nDetect']
test['nO-nD'] = test['nObserve'] - test['nDetect']

In [10]:
# 단순 나눗셈
train['u/dered_u'] = train['u'] / train['dered_u'] - 1
train['g/dered_g'] = train['g'] / train['dered_g'] - 1
train['r/dered_r'] = train['r'] / train['dered_r'] - 1
train['i/dered_i'] = train['i'] / train['dered_i'] - 1
train['z/dered_z'] = train['z'] / train['dered_z'] - 1

test['u/dered_u'] = test['u'] / test['dered_u'] - 1
test['g/dered_g'] = test['g'] / test['dered_g'] - 1
test['r/dered_r'] = test['r'] / test['dered_r'] - 1
test['i/dered_i'] = test['i'] / test['dered_i'] - 1
test['z/dered_z'] = test['z'] / test['dered_z'] - 1

In [11]:
train['M_u'] = train['u'] - 5*(np.log(train['distance']) - 1)
train['M_g'] = train['g'] - 5*(np.log(train['distance']) - 1)
train['M_r'] = train['r'] - 5*(np.log(train['distance']) - 1)
train['M_i'] = train['i'] - 5*(np.log(train['distance']) - 1)
train['M_z'] = train['z'] - 5*(np.log(train['distance']) - 1)
train['M'] = train['rare_max'] - 5*(np.log(train['distance']) - 1)
train['M_dered_u'] = train['dered_u'] - 5*(np.log(train['distance']) - 1)
train['M_dered_g'] = train['dered_g'] - 5*(np.log(train['distance']) - 1)
train['M_dered_r'] = train['dered_r'] - 5*(np.log(train['distance']) - 1)
train['M_dered_i'] = train['dered_i'] - 5*(np.log(train['distance']) - 1)
train['M_dered_z'] = train['dered_z'] - 5*(np.log(train['distance']) - 1)
train['dered_M'] = train['dered_max'] - 5*(np.log(train['distance']) - 1)

test['M_u'] = test['u'] - 5*(np.log(test['distance']) - 1)
test['M_g'] = test['g'] - 5*(np.log(test['distance']) - 1)
test['M_r'] = test['r'] - 5*(np.log(test['distance']) - 1)
test['M_i'] = test['i'] - 5*(np.log(test['distance']) - 1)
test['M_z'] = test['z'] - 5*(np.log(test['distance']) - 1)
test['M'] = test['rare_max'] - 5*(np.log(test['distance']) - 1)
test['M_dered_u'] = test['dered_u'] - 5*(np.log(test['distance']) - 1)
test['M_dered_g'] = test['dered_g'] - 5*(np.log(test['distance']) - 1)
test['M_dered_r'] = test['dered_r'] - 5*(np.log(test['distance']) - 1)
test['M_dered_i'] = test['dered_i'] - 5*(np.log(test['distance']) - 1)
test['M_dered_z'] = test['dered_z'] - 5*(np.log(test['distance']) - 1)
test['dered_M'] = test['dered_max'] - 5*(np.log(test['distance']) - 1)

In [12]:
Y = train['class']
del train['class']
X = train

In [14]:
# split data into train and test sets

X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=7)
eval_set = [(X_val, y_val)]

In [ ]:
def objective(params):
    time1 = time.time()
    params = {
        'max_depth': int(params['max_depth']),
        'gamma': "{:.3f}".format(params['gamma']),
        'subsample': "{:.2f}".format(params['subsample']),
        'reg_alpha': "{:.3f}".format(params['reg_alpha']),
        'reg_lambda': "{:.3f}".format(params['reg_lambda']),
        'learning_rate': "{:.3f}".format(params['learning_rate']),
        'num_leaves': '{:.3f}'.format(params['num_leaves']),
        'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
        'min_child_samples': '{:.3f}'.format(params['min_child_samples']),
        'feature_fraction': '{:.3f}'.format(params['feature_fraction']),
        'bagging_fraction': '{:.3f}'.format(params['bagging_fraction'])
    }

    print("\n############## New Run ################")
    print(f"params = {params}")
    FOLDS = 5
    count=1
    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)

    tss = TimeSeriesSplit(n_splits=FOLDS)
    y_preds = np.zeros(submission.shape[0])
    y_oof = np.zeros(X_train.shape[0])
    score_mean = 0
    for tr_idx, val_idx in tss.split(X_train, y_train):
        clf = xgb.XGBClassifier(
            objective='multi:softmax',
            num_class=3,
            eval_metric='merror',
            n_estimators=1000,
            random_state=4,
            verbose=True, 
            tree_method='gpu_hist', 
            **params
        )

        X_tr, X_vl = X_train.iloc[tr_idx, :], X_train.iloc[val_idx, :]
        y_tr, y_vl = y_train.iloc[tr_idx], y_train.iloc[val_idx]
        
        clf.fit(X_tr, y_tr)
        #y_pred_train = clf.predict_proba(X_vl)[:,1]
        #print(y_pred_train)
        score = make_scorer(accuracy_score, needs_proba=True)(clf, X_vl, y_vl)
        # plt.show()
        score_mean += score
        print(f'{count} CV - score: {round(score, 4)}')
        count += 1
    time2 = time.time() - time1
    print(f"Total Time Run: {round(time2 / 60,2)}")
    gc.collect()
    print(f'Mean accuracy: {score_mean / FOLDS}')
    del X_tr, X_vl, y_tr, y_vl, clf, score
    return -(score_mean / FOLDS)


space = {
    # The maximum depth of a tree, same as GBM.
    # Used to control over-fitting as higher depth will allow model 
    # to learn relations very specific to a particular sample.
    # Should be tuned using CV.
    # Typical values: 3-10
    'max_depth': hp.quniform('max_depth', 10, 23, 1),
    
    # reg_alpha: L1 regularization term. L1 regularization encourages sparsity 
    # (meaning pulling weights to 0). It can be more useful when the objective
    # is logistic regression since you might need help with feature selection.
    'reg_alpha':  hp.uniform('reg_alpha', 0.01, 0.4),
    
    # reg_lambda: L2 regularization term. L2 encourages smaller weights, this
    # approach can be more useful in tree-models where zeroing 
    # features might not make much sense.
    'reg_lambda': hp.uniform('reg_lambda', 0.01, .4),
    
    # eta: Analogous to learning rate in GBM
    # Makes the model more robust by shrinking the weights on each step
    # Typical final values to be used: 0.01-0.2
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.1),
    
    # colsample_bytree: Similar to max_features in GBM. Denotes the 
    # fraction of columns to be randomly samples for each tree.
    # Typical values: 0.5-1
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, .9),
    
    # A node is split only when the resulting split gives a positive
    # reduction in the loss function. Gamma specifies the 
    # minimum loss reduction required to make a split.
    # Makes the algorithm conservative. The values can vary depending on the loss function and should be tuned.
    'gamma': hp.uniform('gamma', 0.01, .7),
    
    # more increases accuracy, but may lead to overfitting.
    # num_leaves: the number of leaf nodes to use. Having a large number 
    # of leaves will improve accuracy, but will also lead to overfitting.
    'num_leaves': hp.choice('num_leaves', list(range(20, 250, 10))),
    
    # specifies the minimum samples per leaf node.
    # the minimum number of samples (data) to group into a leaf. 
    # The parameter can greatly assist with overfitting: larger sample
    # sizes per leaf will reduce overfitting (but may lead to under-fitting).
    'min_child_samples': hp.choice('min_child_samples', list(range(100, 250, 10))),
    
    # subsample: represents a fraction of the rows (observations) to be 
    # considered when building each subtree. Tianqi Chen and Carlos Guestrin
    # in their paper A Scalable Tree Boosting System recommend 
    'subsample': hp.choice('subsample', [0.2, 0.4, 0.5, 0.6, 0.7, .8, .9]),
    
    # randomly select a fraction of the features.
    # feature_fraction: controls the subsampling of features used
    # for training (as opposed to subsampling the actual training data in 
    # the case of bagging). Smaller fractions reduce overfitting.
    'feature_fraction': hp.uniform('feature_fraction', 0.4, .8),
    
    # randomly bag or subsample training data.
    'bagging_fraction': hp.uniform('bagging_fraction', 0.4, .9)
    
    # bagging_fraction and bagging_freq: enables bagging (subsampling) 
    # of the training data. Both values need to be set for bagging to be used.
    # The frequency controls how often (iteration) bagging is used. Smaller
    # fractions and frequencies reduce overfitting.
}

In [ ]:
# Set algoritm parameters
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=27)

# Print best parameters
best_params = space_eval(space, best)

In [ ]:
print("BEST PARAMS: ", best_params)

best_params['max_depth'] = int(best_params['max_depth'])

In [ ]:
# https://www.kaggle.com/kabure/extensive-eda-and-modeling-xgb-hyperopt